# Tutorial 15: FastMCP Server Basics

## Building Custom MCP Servers for Enterprise AI Agents

### What You'll Learn

In this tutorial, you'll learn how to build a **FastMCP server** that exposes custom tools to AI agents using the Model Context Protocol (MCP). This is the foundation for building enterprise-grade, scalable AI agent architectures.

**Key Concepts:**
- Model Context Protocol (MCP) overview
- Creating a FastMCP server
- Defining custom tools with schemas
- Testing MCP servers locally
- Integrating MCP servers with AI agents

**Use Case:** Building a Travel Booking MCP Server with flight search, hotel availability, and currency conversion tools.

**Duration:** 45 minutes

---

### Prerequisites

- Completed tutorials 01-14
- Python 3.11+
- Understanding of REST APIs
- Azure OpenAI or OpenAI API access

## Part 1: Understanding Model Context Protocol (MCP)

### What is MCP?

**Model Context Protocol (MCP)** is an open standard for connecting AI assistants to external data sources and tools. Think of it as a standardized way for AI agents to:

- Access external data (databases, APIs, file systems)
- Execute functions and tools
- Maintain context across interactions
- Scale agent capabilities without modifying agent code

### Why FastMCP?

**FastMCP** is a Python framework that makes it easy to build MCP servers with:
- Simple, decorator-based tool definitions
- Automatic schema generation
- Built-in validation
- Easy local testing
- Production-ready server implementation

### Architecture

```
┌─────────────────┐
│   AI Agent      │
│  (OpenAI/Azure) │
└────────┬────────┘
         │ MCP Protocol
         │
┌────────┴────────┐
│  FastMCP Server │
│  ┌───────────┐  │
│  │  Tool 1   │  │
│  │  Tool 2   │  │
│  │  Tool 3   │  │
│  └───────────┘  │
└─────────────────┘
```

## Part 2: Setup and Installation

First, let's install the required packages for building our FastMCP server.

In [ ]:
# Install FastMCP and required dependencies
# Using latest compatible versions
%pip install fastmcp uvicorn python-dotenv

  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
Using cached referencing-0.36.2-py3-none-any.whl (26 kB)
Using cached referencing-0.36.2-py3-none-any.whl (26 kB)


In [ ]:
# Import required libraries
import os
from datetime import datetime, timedelta
from typing import Optional, List
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv()

print("Environment loaded successfully")

Environment loaded successfully


## Part 3: Creating Your First FastMCP Server

Let's build a Travel Booking MCP Server with three tools:
1. **search_flights** - Search for available flights
2. **check_hotel_availability** - Check hotel room availability
3. **convert_currency** - Convert between currencies

We'll create a file called `travel_mcp_server.py` that contains our MCP server implementation.

In [ ]:
# Create the MCP server file
server_code = '''
"""
Travel Booking MCP Server
A FastMCP server that provides travel-related tools for AI agents
"""

from fastmcp import FastMCP
from datetime import datetime, timedelta
from typing import List, Optional, Dict
from pydantic import BaseModel, Field
import random

# Initialize the MCP server
mcp = FastMCP("Travel Booking Server")

# Data models for structured responses
class Flight(BaseModel):
    """Flight information"""
    flight_number: str
    airline: str
    departure: str
    arrival: str
    departure_time: str
    arrival_time: str
    price: float
    currency: str = "USD"
    available_seats: int

class Hotel(BaseModel):
    """Hotel information"""
    hotel_name: str
    location: str
    room_type: str
    price_per_night: float
    currency: str = "USD"
    available_rooms: int
    amenities: List[str]

# Underlying functions that return JSON-serializable data
def _search_flights(
    origin: str,
    destination: str,
    departure_date: str,
    passengers: int = 1
) -> List[Dict]:
    """
    Search for available flights between two cities.
    
    Args:
        origin: Departure city (e.g., "New York", "NYC")
        destination: Arrival city (e.g., "London", "LHR")
        departure_date: Date in YYYY-MM-DD format
        passengers: Number of passengers (default: 1)
    
    Returns:
        List of available flights with details (as dicts)
    """
    # Simulate flight search (in production, call real APIs)
    airlines = ["Delta", "United", "American", "British Airways"]
    
    flights = []
    for i in range(3):
        dep_time = datetime.strptime(departure_date, "%Y-%m-%d")
        dep_time = dep_time.replace(hour=random.randint(6, 20), minute=random.choice([0, 15, 30, 45]))
        arr_time = dep_time + timedelta(hours=random.randint(6, 12))
        
        flight = {
            "flight_number": f"{random.choice(['DL', 'UA', 'AA', 'BA'])}{random.randint(100, 999)}",
            "airline": random.choice(airlines),
            "departure": origin,
            "arrival": destination,
            "departure_time": dep_time.strftime("%Y-%m-%d %H:%M"),
            "arrival_time": arr_time.strftime("%Y-%m-%d %H:%M"),
            "price": round(random.uniform(300, 1200), 2),
            "currency": "USD",
            "available_seats": random.randint(5, 50)
        }
        flights.append(flight)
    
    return flights

def _check_hotel_availability(
    location: str,
    check_in: str,
    check_out: str,
    guests: int = 2
) -> List[Dict]:
    """
    Check hotel room availability in a specific location.
    
    Args:
        location: City or area name
        check_in: Check-in date in YYYY-MM-DD format
        check_out: Check-out date in YYYY-MM-DD format
        guests: Number of guests (default: 2)
    
    Returns:
        List of available hotels with details (as dicts)
    """
    # Simulate hotel search (in production, call real APIs)
    hotel_names = ["Grand Plaza Hotel", "Seaside Resort", "Downtown Suites", "Airport Inn"]
    room_types = ["Standard Room", "Deluxe Room", "Suite", "Executive Suite"]
    
    hotels = []
    for i in range(4):
        hotel = {
            "hotel_name": random.choice(hotel_names),
            "location": location,
            "room_type": random.choice(room_types),
            "price_per_night": round(random.uniform(80, 350), 2),
            "currency": "USD",
            "available_rooms": random.randint(1, 10),
            "amenities": random.sample(["WiFi", "Pool", "Gym", "Restaurant", "Spa", "Parking"], k=3)
        }
        hotels.append(hotel)
    
    return hotels

def _convert_currency(
    amount: float,
    from_currency: str,
    to_currency: str
) -> Dict:
    """
    Convert amount from one currency to another.
    
    Args:
        amount: Amount to convert
        from_currency: Source currency code (e.g., "USD", "EUR", "GBP")
        to_currency: Target currency code
    
    Returns:
        Dictionary with conversion details
    """
    # Simulate currency conversion (in production, call real exchange rate API)
    exchange_rates = {
        "USD": {"EUR": 0.85, "GBP": 0.73, "JPY": 110.0, "USD": 1.0},
        "EUR": {"USD": 1.18, "GBP": 0.86, "JPY": 129.0, "EUR": 1.0},
        "GBP": {"USD": 1.37, "EUR": 1.16, "JPY": 150.0, "GBP": 1.0},
        "JPY": {"USD": 0.0091, "EUR": 0.0078, "GBP": 0.0067, "JPY": 1.0}
    }
    
    rate = exchange_rates.get(from_currency, {}).get(to_currency, 1.0)
    converted_amount = round(amount * rate, 2)
    
    return {
        "original_amount": amount,
        "original_currency": from_currency,
        "converted_amount": converted_amount,
        "converted_currency": to_currency,
        "exchange_rate": rate,
        "timestamp": datetime.now().isoformat()
    }

def _get_server_info() -> Dict:
    """Get information about this MCP server"""
    return {
        "name": "Travel Booking Server",
        "version": "1.0.0",
        "tools": ["search_flights", "check_hotel_availability", "convert_currency"],
        "description": "MCP server for travel booking operations"
    }

# MCP tool wrappers - convert results to Pydantic models for MCP
@mcp.tool()
def search_flights(
    origin: str,
    destination: str,
    departure_date: str,
    passengers: int = 1
) -> List[Flight]:
    """
    Search for available flights between two cities.
    
    Args:
        origin: Departure city (e.g., "New York", "NYC")
        destination: Arrival city (e.g., "London", "LHR")
        departure_date: Date in YYYY-MM-DD format
        passengers: Number of passengers (default: 1)
    
    Returns:
        List of available flights with details
    """
    flights_data = _search_flights(origin, destination, departure_date, passengers)
    return [Flight(**f) for f in flights_data]

@mcp.tool()
def check_hotel_availability(
    location: str,
    check_in: str,
    check_out: str,
    guests: int = 2
) -> List[Hotel]:
    """
    Check hotel room availability in a specific location.
    
    Args:
        location: City or area name
        check_in: Check-in date in YYYY-MM-DD format
        check_out: Check-out date in YYYY-MM-DD format
        guests: Number of guests (default: 2)
    
    Returns:
        List of available hotels with details
    """
    hotels_data = _check_hotel_availability(location, check_in, check_out, guests)
    return [Hotel(**h) for h in hotels_data]

@mcp.tool()
def convert_currency(
    amount: float,
    from_currency: str,
    to_currency: str
) -> dict:
    """
    Convert amount from one currency to another.
    
    Args:
        amount: Amount to convert
        from_currency: Source currency code (e.g., "USD", "EUR", "GBP")
        to_currency: Target currency code
    
    Returns:
        Dictionary with conversion details
    """
    return _convert_currency(amount, from_currency, to_currency)

@mcp.tool()
def get_server_info() -> dict:
    """Get information about this MCP server"""
    return _get_server_info()

# ASGI app for uvicorn (used in container deployment - Tutorial 16)
app = mcp.http_app()

if __name__ == "__main__":
    # Run the server locally via stdio transport
    mcp.run()
'''

# Write the server code to a file
with open('travel_mcp_server.py', 'w') as f:
    f.write(server_code)

print("MCP Server created: travel_mcp_server.py")
print("Tools available: search_flights, check_hotel_availability, convert_currency, get_server_info")
print("\nNote: Exported _search_flights, _check_hotel_availability, _convert_currency functions")
print("      return JSON-serializable dicts for use with Azure AI Agent Framework")


MCP Server created: travel_mcp_server.py
Tools available: search_flights, check_hotel_availability, convert_currency, get_server_info

Note: Exported _search_flights, _check_hotel_availability, _convert_currency functions
      return JSON-serializable dicts for use with Azure AI Agent Framework


## Part 4: Testing the MCP Server Locally

Now let's test our MCP server locally. We'll:
1. Start the server in the background
2. Test individual tools
3. Verify the responses

In [ ]:
# Test the tools by importing and calling the underlying functions
# Need to reload the module if it was already imported
import sys
if 'travel_mcp_server' in sys.modules:
    import importlib
    import travel_mcp_server
    importlib.reload(travel_mcp_server)
    from travel_mcp_server import _search_flights, _check_hotel_availability, _convert_currency, _get_server_info
else:
    from travel_mcp_server import _search_flights, _check_hotel_availability, _convert_currency, _get_server_info

# Test 1: Search for flights
print("=== Testing Flight Search ===")
flights = _search_flights(
    origin="New York",
    destination="London",
    departure_date="2025-11-15",
    passengers=2
)

for flight in flights:
    print(f"\n{flight['airline']} {flight['flight_number']}")
    print(f"  {flight['departure']} → {flight['arrival']}")
    print(f"  Departs: {flight['departure_time']}")
    print(f"  Arrives: {flight['arrival_time']}")
    print(f"  Price: ${flight['price']} ({flight['available_seats']} seats available)")


=== Testing Flight Search ===

Delta AA234
  New York → London
  Departs: 2025-11-15 16:30
  Arrives: 2025-11-16 03:30
  Price: $1055.11 (14 seats available)

United UA712
  New York → London
  Departs: 2025-11-15 09:00
  Arrives: 2025-11-15 20:00
  Price: $1051.78 (37 seats available)

United AA327
  New York → London
  Departs: 2025-11-15 06:45
  Arrives: 2025-11-15 13:45
  Price: $721.31 (8 seats available)


In [ ]:
# Test 2: Check hotel availability
print("\n\n=== Testing Hotel Search ===")
hotels = _check_hotel_availability(
    location="London",
    check_in="2025-11-15",
    check_out="2025-11-18",
    guests=2
)

for hotel in hotels:
    print(f"\n{hotel['hotel_name']}")
    print(f"  Location: {hotel['location']}")
    print(f"  Room Type: {hotel['room_type']}")
    print(f"  Price: ${hotel['price_per_night']}/night ({hotel['available_rooms']} rooms available)")
    print(f"  Amenities: {', '.join(hotel['amenities'])}")




=== Testing Hotel Search ===

Grand Plaza Hotel
  Location: London
  Room Type: Suite
  Price: $242.4/night (9 rooms available)
  Amenities: Parking, Spa, WiFi

Downtown Suites
  Location: London
  Room Type: Standard Room
  Price: $158.03/night (1 rooms available)
  Amenities: Spa, Parking, WiFi

Grand Plaza Hotel
  Location: London
  Room Type: Suite
  Price: $230.93/night (3 rooms available)
  Amenities: Spa, Restaurant, Pool

Seaside Resort
  Location: London
  Room Type: Executive Suite
  Price: $165.93/night (5 rooms available)
  Amenities: Pool, Parking, Spa


In [ ]:
# Test 3: Convert currency
print("\n\n=== Testing Currency Conversion ===")
conversion = _convert_currency(
    amount=1000.0,
    from_currency="USD",
    to_currency="EUR"
)

print(f"\n{conversion['original_amount']} {conversion['original_currency']} = "
      f"{conversion['converted_amount']} {conversion['converted_currency']}")
print(f"Exchange Rate: {conversion['exchange_rate']}")
print(f"Timestamp: {conversion['timestamp']}")




=== Testing Currency Conversion ===

1000.0 USD = 850.0 EUR
Exchange Rate: 0.85
Timestamp: 2025-12-02T09:29:15.259765


## Part 5: Integrating MCP Server with Azure AI Agent

Now let's build an Azure AI Agent that can use our MCP server tools. We'll use the Azure AI Agent Framework that we've been using throughout this tutorial series.

In [ ]:
# Import Azure AI Agent Framework
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity import DefaultAzureCredential

print("Azure AI Agent Framework imported successfully")

Azure AI Agent Framework imported successfully


In [ ]:
# Create Azure AI Agent with MCP tools
from azure.identity.aio import AzureCliCredential

# Verify environment variables are set
project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model_deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

if not project_endpoint or not model_deployment:
    print("WARNING: Missing required environment variables:")
    print("   Please set in your .env file:")
    print("   - AZURE_AI_PROJECT_ENDPOINT")
    print("   - AZURE_AI_MODEL_DEPLOYMENT_NAME")
    print("\n   See Tutorial 01b for Azure AI Foundry setup")
else:
    print("Azure AI configuration loaded")
    print(f"  Endpoint: {project_endpoint}")
    print(f"  Model: {model_deployment}")
    print("\nAgent will be created with MCP tools:")
    print("  - search_flights")
    print("  - check_hotel_availability") 
    print("  - convert_currency")


Azure AI configuration loaded
  Endpoint: https://gk-agent-framework-project.services.ai.azure.com/api/projects/agentframworkProject
  Model: gpt-4.1

Agent will be created with MCP tools:
  - search_flights
  - check_hotel_availability
  - convert_currency


In [ ]:
# Test the agent with a simple query
import asyncio

async def test_agent_simple():
    """Test the agent with a simple travel query"""
    print("=" * 70)
    print("TEST 1: Simple Flight Search (Local Tools)")
    print("=" * 70)
    
    user_query = "I need to find flights from New York to London on November 15th for 2 passengers."
    
    print(f"\nUser: {user_query}\n")
    
    # Create and use the agent with Azure AI
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            # Reads AZURE_AI_PROJECT_ENDPOINT and AZURE_AI_MODEL_DEPLOYMENT_NAME from env
        ) as chat_client,
    ):
        # Create agent with local Python function tools
        agent = chat_client.create_agent(
            name="TravelMCPAgent",
            instructions="""
            You are a helpful travel assistant with access to real-time travel booking tools.
            
            You can:
            - Search for flights between cities
            - Check hotel availability and pricing
            - Convert currencies for budget planning
            
            When helping users plan trips:
            1. Use the available tools to get accurate, real-time information
            2. Provide clear, organized recommendations
            3. Help with currency conversions when discussing costs
            
            Be friendly, professional, and detail-oriented.
            """,
            tools=[_search_flights, _check_hotel_availability, _convert_currency]
        )
        
        # Run the agent
        response = await agent.run(user_query)
        
        print(f"Agent: {response.text}\n")
    
    print("=" * 70)
    print("Note: This uses LOCAL Python functions as tools")
    print("      See Tutorial 16 for deploying as a remote MCP server")

# Run the test only if environment is configured
if project_endpoint and model_deployment:
    await test_agent_simple()
else:
    print("WARNING: Please configure Azure AI environment variables first")
    print("   See cell above for required configuration")

TEST 1: Simple Flight Search (Local Tools)

User: I need to find flights from New York to London on November 15th for 2 passengers.

Agent: Here are some available flights from New York to London on November 15th for 2 passengers:

1. American Airlines (AA742)
   - Departure: 13:00
   - Arrival: 20:00
   - Price: $306.02 per person

2. Delta Airlines (AA289)
   - Departure: 12:00
   - Arrival: 23:00
   - Price: $526.66 per person

3. Delta Airlines (BA575)
   - Departure: 07:15
   - Arrival: 18:15
   - Price: $532.02 per person

Would you like more details about any of these flights or want help with booking?

Agent: Here are some available flights from New York to London on November 15th for 2 passengers:

1. American Airlines (AA742)
   - Departure: 13:00
   - Arrival: 20:00
   - Price: $306.02 per person

2. Delta Airlines (AA289)
   - Departure: 12:00
   - Arrival: 23:00
   - Price: $526.66 per person

3. Delta Airlines (BA575)
   - Departure: 07:15
   - Arrival: 18:15
   - Price: 

In [ ]:
# Test the agent with a complex multi-tool query
async def test_agent_complex():
    """Test the agent with a complex query requiring multiple tools"""
    print("\n" + "=" * 70)
    print("TEST 2: Complex Multi-Tool Query (Local Tools)")
    print("=" * 70)
    
    user_query = """
    I'm planning a trip to London from New York. Can you help me with:
    1. Flights for November 15th for 2 passengers
    2. Hotels for 3 nights (Nov 15-18)
    3. Total cost estimate in Euros if my budget is $2000 USD
    """
    
    print(f"\nUser: {user_query}\n")
    
    # Create and use the agent with Azure AI
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
        ) as chat_client,
    ):
        # Create agent with local Python function tools
        agent = chat_client.create_agent(
            name="TravelMCPAgent",
            instructions="""
            You are a helpful travel assistant with access to real-time travel booking tools.
            Use multiple tools as needed to provide comprehensive travel planning assistance.
            """,
            tools=[_search_flights, _check_hotel_availability, _convert_currency]
        )
        
        # Run the agent - it will automatically use multiple tools as needed
        response = await agent.run(user_query)
        
        print(f"Agent: {response.text}\n")
    
    print("=" * 70)
    print("Note: This uses LOCAL Python functions as tools")
    print("      Tools run in the same process as the agent")

# Run the complex test only if environment is configured
if project_endpoint and model_deployment:
    await test_agent_complex()
else:
    print("WARNING: Please configure Azure AI environment variables first")


TEST 2: Complex Multi-Tool Query (Local Tools)

User: 
    I'm planning a trip to London from New York. Can you help me with:
    1. Flights for November 15th for 2 passengers
    2. Hotels for 3 nights (Nov 15-18)
    3. Total cost estimate in Euros if my budget is $2000 USD
    



[2025-11-30 22:03:58 - /Users/gk/AIFoundryProjects/agent-framework-tutorials/venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


Agent: Here are the details for your trip to London from New York:

1. Flight Options (Nov 15th for 2 passengers):
- Delta (DL224): Departs 12:30, arrives 22:30. Price per ticket: $735.30 USD.
- United (BA196): Departs 16:45, arrives next day 02:45. Price per ticket: $308.56 USD (best value).
- American (AA949): Departs 13:45, arrives 21:45. Price per ticket: $521.69 USD.

2. Hotel Options in London (Nov 15-18, 3 nights for 2 guests):
- Grand Plaza Hotel (Deluxe Room): $213.19/night
- Downtown Suites (Standard Room): $226.83/night
- Seaside Resort (Deluxe Room): $250.51/night
- Seaside Resort (Executive Suite): $344.54/night

3. Budget Conversion:
- Your total budget of $2000 USD is approximately €1700 EUR at today's exchange rate (1 USD ≈ 0.85 EUR).

Cost Estimate for your trip (if choosing best-value options):
- Flights (United BA196): 2 x $308.56 = $617.12 USD
- Hotel (Grand Plaza Hotel): 3 x $213.19 = $639.57 USD
- Estimated Total: $617.12 + $639.57 = $1256.69 USD
- In Euros: ≈ €10

## Part 5b: Hosted MCP Server Pattern (Preview)

This section demonstrates how to **run the MCP server locally** and explains the `HostedMCPTool` pattern for production.

**Important:** `HostedMCPTool` requires a **publicly accessible URL** because Azure AI Agent Service (running in Azure) needs to reach the MCP server. For local development:
- ✅ Use **local Python functions** as tools (what we did in Part 5)
- ✅ Test the MCP server locally with direct HTTP calls
- ❌ `HostedMCPTool` won't work with `localhost` (Azure can't reach it)

**For production with `HostedMCPTool`:**
```
┌─────────────────┐     HTTP/SSE      ┌─────────────────────────┐
│  Azure AI Agent │ ◄───────────────► │  MCP Server on Azure    │
│  Service        │    (public URL)   │  (Container Apps/AKS)   │
└─────────────────┘                   └─────────────────────────┘
```

See **Tutorial 16** for deploying to Azure Container Apps.

In [ ]:
# Start the MCP server locally and test it with HTTP requests
# This verifies the server works before deploying to Azure

import subprocess
import time
import requests

# Start the MCP server as a background process
MCP_SERVER_PORT = 8765
MCP_SERVER_URL = f"http://localhost:{MCP_SERVER_PORT}"

print(f"Starting MCP server on {MCP_SERVER_URL}...")

# Run uvicorn in background - it will serve the ASGI app from travel_mcp_server.py
server_process = subprocess.Popen(
    ["uvicorn", "travel_mcp_server:app", "--host", "0.0.0.0", "--port", str(MCP_SERVER_PORT)],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    cwd=os.getcwd()
)

# Wait for server to start
time.sleep(3)

# Check if server is running
if server_process.poll() is None:
    print(f"✅ MCP Server started successfully (PID: {server_process.pid})")
    print(f"   URL: {MCP_SERVER_URL}")
    print(f"   SSE Endpoint: {MCP_SERVER_URL}/sse")
    
    # Test the MCP server endpoints
    print("\n--- Testing MCP Server Endpoints ---")
    
    # Test the SSE endpoint (should return 200 for GET, starts SSE stream)
    try:
        # Use a short timeout since SSE will stream indefinitely
        response = requests.get(f"{MCP_SERVER_URL}/sse", timeout=2, stream=True)
        print(f"✅ SSE endpoint (/sse): {response.status_code} - Ready for MCP connections")
        response.close()
    except requests.exceptions.ReadTimeout:
        print(f"✅ SSE endpoint (/sse): Connected (streaming mode)")
    except Exception as e:
        print(f"❌ SSE endpoint test: {e}")
    
    # Test the MCP message endpoint (POST)
    try:
        response = requests.post(f"{MCP_SERVER_URL}/mcp/", json={}, timeout=5)
        print(f"✅ MCP endpoint (/mcp/): {response.status_code}")
    except Exception as e:
        print(f"⚠️  MCP endpoint: {type(e).__name__} (expected - requires valid MCP message)")
    
    print(f"\n📡 Server is running and ready for connections!")
    print(f"   To connect with an MCP client, use: {MCP_SERVER_URL}/sse")
else:
    stdout, stderr = server_process.communicate()
    print(f"❌ Server failed to start:")
    print(f"   stdout: {stdout.decode()}")
    print(f"   stderr: {stderr.decode()}")

Starting MCP server on http://localhost:8765...


NameError: name 'os' is not defined

In [ ]:
# Demonstrate HostedMCPTool syntax (for when server is deployed to Azure)
# NOTE: This pattern requires a PUBLIC URL - won't work with localhost!

from agent_framework import HostedMCPTool

# This is how you would use HostedMCPTool with a deployed server:
print("=" * 70)
print("HostedMCPTool Pattern (for Production)")
print("=" * 70)

# Example: Creating a HostedMCPTool for a deployed MCP server
# Replace with your actual Azure Container Apps URL after deploying (Tutorial 16)
DEPLOYED_MCP_URL = "https://your-mcp-server.azurecontainerapps.io/sse"

print(f"""
When your MCP server is deployed to Azure, use HostedMCPTool:

```python
from agent_framework import HostedMCPTool

mcp_tool = HostedMCPTool(
    name="Travel Booking MCP Server",
    url="{DEPLOYED_MCP_URL}",
    approval_mode="never_require"
)

agent = chat_client.create_agent(
    name="TravelAgent",
    instructions="...",
    tools=mcp_tool  # Use hosted MCP instead of local functions
)
```

The agent will then call tools on the remote MCP server over HTTP/SSE.
""")

print("=" * 70)
print("For local development, use local Python functions (Part 5)")
print("For production, deploy to Azure Container Apps (Tutorial 16)")
print("=" * 70)

HostedMCPTool Pattern (for Production)

When your MCP server is deployed to Azure, use HostedMCPTool:

```python
from agent_framework import HostedMCPTool

mcp_tool = HostedMCPTool(
    name="Travel Booking MCP Server",
    url="https://your-mcp-server.azurecontainerapps.io/sse",
    approval_mode="never_require"
)

agent = chat_client.create_agent(
    name="TravelAgent",
    instructions="...",
    tools=mcp_tool  # Use hosted MCP instead of local functions
)
```

The agent will then call tools on the remote MCP server over HTTP/SSE.

For local development, use local Python functions (Part 5)
For production, deploy to Azure Container Apps (Tutorial 16)


In [ ]:
# Clean up: Stop the MCP server when done
print("Stopping MCP server...")

if server_process and server_process.poll() is None:
    server_process.terminate()
    server_process.wait(timeout=5)
    print(f"✅ MCP Server stopped (PID: {server_process.pid})")
else:
    print("Server was not running")

# Note: In production, the MCP server would run continuously
# and be deployed to Azure Container Apps (see Tutorial 16)

Stopping MCP server...
Server was not running


## Understanding Local vs Hosted MCP Tools

### What We Just Built

In this tutorial, we used **local Python functions as tools**:
```python
# Tools are Python functions in the same process
agent = chat_client.create_agent(
    tools=[_search_flights, _check_hotel_availability, _convert_currency]
)
```

**This approach is perfect for:**
- Development and testing
- Quick prototyping
- Single-agent applications
- Learning MCP concepts

### What's Next: Hosted MCP Servers

In **Tutorial 16**, you'll deploy this MCP server to Azure and use **HostedMCPTool**:
```python
# Tools are hosted on a remote MCP server
agent = chat_client.create_agent(
    tools=HostedMCPTool(
        name="Travel Booking MCP Server",
        url="https://your-app.azurecontainerapps.io",
        approval_mode="never_require"
    )
)
```

**Hosted MCP is better for:**
- Production deployments
- Multi-agent systems sharing tools
- Tools requiring isolation or independent scaling
- Enterprise applications with security requirements

### Key Difference

| Aspect | Local Tools (Tutorial 15) | Hosted MCP (Tutorial 16) |
|--------|---------------------------|--------------------------|
| **Location** | Same process as agent | Remote HTTP server |
| **Performance** | Fast (no network) | Network latency |
| **Scaling** | Scales with agent | Independent scaling |
| **Sharing** | One agent only | Multiple agents |
| **Deployment** | Simple | Requires infrastructure |
| **Use Case** | Development/Testing | Production |


## Part 6: Best Practices for MCP Server Development

### Critical Lessons Learned

**1. Dual Function Pattern (Required for Agent Integration)**
```python
# Underscore functions return JSON-serializable dicts (for agents)
def _search_flights(...) -> List[Dict]:
    return [{"flight_number": "AA123", ...}]

# MCP wrappers convert to Pydantic models (for MCP server)
@mcp.tool()
def search_flights(...) -> List[Flight]:
    return [Flight(**f) for f in _search_flights(...)]
```

**2. Package Management**
- Don't pin FastMCP versions - use `pip install fastmcp` without constraints
- Let pip auto-resolve dependencies (FastMCP evolves rapidly)

**3. Environment & Security**
- Store credentials in `.env` files
- Validate environment variables before agent creation
- Never hardcode sensitive values

**4. Testing & Error Handling**
- Test underscore functions locally first
- Test with agents to verify JSON serialization
- Wrap tool functions in try-except blocks
- Return structured error responses

**5. Production Considerations**
- Make tools async for I/O operations
- Implement rate limiting and timeouts
- Use caching for frequently requested data
- Monitor tool invocations and error rates

## Summary and Key Takeaways

### What You've Learned

1. **Model Context Protocol (MCP)**: A standardized way to connect AI agents to external tools and data sources

2. **FastMCP Framework**: Python framework for building MCP servers with:
   - Decorator-based tool definitions
   - Automatic schema generation
   - Built-in validation

3. **Tool Development**: How to create MCP tools with:
   - Clear function signatures
   - Pydantic models for structured data
   - Proper documentation

4. **Agent Integration**: How to connect MCP servers to AI agents using:
   - OpenAI function calling
   - Tool execution handlers
   - Multi-turn conversations

### Next Steps

In the next tutorial (**16_deploy_fastmcp_to_aca.ipynb**), you'll learn how to:
- Dockerize your MCP server
- Deploy to Azure Container Apps
- Configure environment variables and scaling
- Monitor production MCP servers

### Additional Resources

- **FastMCP Documentation**: https://github.com/jlowin/fastmcp
- **MCP Specification**: https://modelcontextprotocol.io
- **Azure Container Apps**: https://learn.microsoft.com/azure/container-apps/

---

**Congratulations!** You've built your first MCP server and integrated it with an AI agent. This is the foundation for building scalable, enterprise-grade AI agent architectures.